In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data

This tutorial shows how to read and decode BBCI data.

## Setup logging to see outputs

In [2]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()

## Load and preprocess data

First set the filename and the sensors you want to load. If you set

```python
load_sensor_names=None
```

or just remove the parameter from the function call, all sensors will be loaded.

In [3]:
from braindecode.datasets.bbci import BBCIDataset
train_filename = '/home/schirrmr/data/BBCI-without-last-runs/BhNoMoSc1S001R01_ds10_1-12.BBCI.mat'
cnt = BBCIDataset(train_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()

Creating RawArray with float64 data, n_channels=3, n_times=3451320
    Range : 0 ... 3451319 =      0.000 ...  6902.638 secs
Ready.


### Preprocessing on continous data

First remove the stimulus channel, than apply any preprocessing you like. There are some very few directions available from Braindecode, such as resample_cnt. But you can apply any function on the chan x time matrix of the mne raw object (`cnt` in the code) by calling `mne_apply` with two arguments:

1. Your function (2d-array-> 2darray), that transforms the channel x timesteps data array
2. the Raw data object from mne itself

In [4]:
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import exponential_running_standardize
# Remove stimulus channel
cnt = cnt.drop_channels(['STI 014'])
cnt = resample_cnt(cnt, 250)
# mne apply will apply the function to the data (a 2d-numpy-array)
# have to transpose data back and forth, since
# exponential_running_standardize expects time x chans order
# while mne object has chans x time order
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)

2017-10-18 14:40:02,301 WARNING : This is not causal, uses future data....
2017-10-18 14:40:02,303 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=1725660
    Range : 0 ... 1725659 =      0.000 ...  6902.636 secs
Ready.


## Transform to epoched dataset 

Braindecode supplies the `create_signal_target_from_raw_mne` function, which will transform the mne raw object into a `SignalAndTarget` object for use in Braindecode.
`name_to_code` should be an `OrderedDict` that maps class names to either one or a list of marker codes for that class.

In [5]:
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from collections import OrderedDict
# can also give lists of marker codes in case a class has multiple marker codes...
name_to_code = OrderedDict([('Right', 1), ('Left', 2), ('Rest', 3), ('Feet', 4)])
segment_ival_ms = [-500,4000]

train_set = create_signal_target_from_raw_mne(cnt, name_to_code, segment_ival_ms)

2017-10-18 14:40:04,600 INFO : Trial per class:
Counter({'Feet': 225, 'Right': 224, 'Rest': 224, 'Left': 224})


## Same for test set

In [6]:
test_filename = '/home/schirrmr/data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat'
cnt = BBCIDataset(test_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()
# Remove stimulus channel
cnt = cnt.drop_channels(['STI 014'])
cnt = resample_cnt(cnt, 250)
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)
test_set = create_signal_target_from_raw_mne(cnt, name_to_code, segment_ival_ms)

Creating RawArray with float64 data, n_channels=3, n_times=617090
    Range : 0 ... 617089 =      0.000 ...  1234.178 secs
Ready.
2017-10-18 14:40:05,669 WARNING : This is not causal, uses future data....
2017-10-18 14:40:05,670 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=308545
    Range : 0 ... 308544 =      0.000 ...  1234.176 secs
Ready.
2017-10-18 14:40:06,236 INFO : Trial per class:
Counter({'Feet': 40, 'Left': 40, 'Rest': 40, 'Right': 40})


<div class="alert alert-info">

In case of start and stop markers, provide a `name_to_stop_codes` dictionary (same as for the start codes in this example) as a final argument to `create_signal_target_from_raw_mne`. See [Read and Decode BBCI Data with Start-Stop-Markers Tutorial](BBCI_Data_Start_Stop.html)


</div>

Split off a validation set.

In [7]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Create the model

In [8]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 800
in_chans = 3
n_classes = 4
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length=30).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

## Setup optimizer and iterator

In [9]:
from torch import optim
import numpy as np

optimizer = optim.Adam(model.parameters())

from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, 3, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

267 predictions per input/trial


## Setup Monitors, Loss function, Stop Criteria

In [10]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression


loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2).squeeze(), targets)

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [11]:
exp.run()

2017-10-18 14:40:09,782 INFO : Run until first stop...
2017-10-18 14:40:10,646 INFO : Epoch 0
2017-10-18 14:40:10,648 INFO : train_loss                7.89184
2017-10-18 14:40:10,649 INFO : valid_loss                7.72731
2017-10-18 14:40:10,650 INFO : test_loss                 7.75617
2017-10-18 14:40:10,650 INFO : train_sample_misclass     0.75013
2017-10-18 14:40:10,651 INFO : valid_sample_misclass     0.74856
2017-10-18 14:40:10,652 INFO : test_sample_misclass      0.75115
2017-10-18 14:40:10,653 INFO : train_misclass            0.75070
2017-10-18 14:40:10,654 INFO : valid_misclass            0.74860
2017-10-18 14:40:10,655 INFO : test_misclass             0.75000
2017-10-18 14:40:10,656 INFO : runtime                   0.00000
2017-10-18 14:40:10,657 INFO : 
2017-10-18 14:40:10,659 INFO : New best valid_misclass: 0.748603
2017-10-18 14:40:10,659 INFO : 
2017-10-18 14:40:11,871 INFO : Time only for training updates: 0.98s
2017-10-18 14:40:12,700 INFO : Epoch 1
2017-10-18 14:40:12

2017-10-18 14:40:29,511 INFO : valid_misclass            0.17318
2017-10-18 14:40:29,512 INFO : test_misclass             0.23125
2017-10-18 14:40:29,512 INFO : runtime                   2.17661
2017-10-18 14:40:29,513 INFO : 
2017-10-18 14:40:30,665 INFO : Time only for training updates: 0.93s
2017-10-18 14:40:31,539 INFO : Epoch 10
2017-10-18 14:40:31,540 INFO : train_loss                0.47951
2017-10-18 14:40:31,541 INFO : valid_loss                0.52164
2017-10-18 14:40:31,542 INFO : test_loss                 0.62324
2017-10-18 14:40:31,543 INFO : train_sample_misclass     0.19307
2017-10-18 14:40:31,543 INFO : valid_sample_misclass     0.24082
2017-10-18 14:40:31,544 INFO : test_sample_misclass      0.29183
2017-10-18 14:40:31,545 INFO : train_misclass            0.12674
2017-10-18 14:40:31,546 INFO : valid_misclass            0.15084
2017-10-18 14:40:31,546 INFO : test_misclass             0.18750
2017-10-18 14:40:31,547 INFO : runtime                   2.14039
2017-10-18 14:

2017-10-18 14:40:49,373 INFO : runtime                   2.07842
2017-10-18 14:40:49,374 INFO : 
2017-10-18 14:40:50,524 INFO : Time only for training updates: 0.94s
2017-10-18 14:40:51,298 INFO : Epoch 20
2017-10-18 14:40:51,300 INFO : train_loss                0.46299
2017-10-18 14:40:51,300 INFO : valid_loss                0.50097
2017-10-18 14:40:51,301 INFO : test_loss                 0.58244
2017-10-18 14:40:51,302 INFO : train_sample_misclass     0.18410
2017-10-18 14:40:51,303 INFO : valid_sample_misclass     0.22857
2017-10-18 14:40:51,304 INFO : test_sample_misclass      0.27696
2017-10-18 14:40:51,304 INFO : train_misclass            0.12396
2017-10-18 14:40:51,305 INFO : valid_misclass            0.12291
2017-10-18 14:40:51,306 INFO : test_misclass             0.16875
2017-10-18 14:40:51,307 INFO : runtime                   1.98218
2017-10-18 14:40:51,308 INFO : 
2017-10-18 14:40:51,310 INFO : New best valid_misclass: 0.122905
2017-10-18 14:40:51,311 INFO : 
2017-10-18 14:4

We arrive at ca. 80% accuracy.

If you want to do trialwise decoding instead of cropped decoding, perform the following changes:


Change:
```python
# This will determine how many crops are processed in parallel
input_time_length = 800
in_chans = 3
n_classes = 4
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length=30).create_network()
```

to:
```python
# This will determine how many crops are processed in parallel
input_time_length = train_set.X.shape[2]
in_chans = 3
n_classes = 4
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length='auto').create_network()
```

Remove:

```python
to_dense_prediction_model(model)
```

Remove:


```python
from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, 3, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))
```

Change:

```python
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)
```

to:

```python
from braindecode.datautil.iterators import BalancedBatchSizeIterator
iterator = BalancedBatchSizeIterator(batch_size=32)
```


Change:

```python
loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2)[:,:,0], targets)
```

to:

```python
loss_function = F.nll_loss
```

Change:

```python
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
```

to:

```python
monitors = [LossMonitor(), MisclassMonitor(col_suffix='misclass'), 
            RuntimeMonitor(),]
```

Resulting code can be seen at [BBCI Data Epoched](BBCI_Data_Epoched.html).